In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.svm import SVR,SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,roc_auc_score,roc_curve,precision_recall_curve,r2_score,mean_squared_error,mean_absolute_error,mean_absolute_percentage_error,root_mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV,RandomizedSearchCV
from pykrige.ok import OrdinaryKriging
import shutil
import graphviz
import dtreeviz


In [ ]:
# 读取数据
data = pd.read_csv(r"F:\cache_data\pre_property_table\dy\feature_ph_dy.csv")
print(len(data))
# 删除有缺失值的行
# data.dropna(inplace=True)
# len(data),data.columns

# 选择数值列并计算它们的均值
numeric_cols = data.select_dtypes(include=[np.number])
means = numeric_cols.mean()
# 使用均值填充每个数值列的缺失值
data[numeric_cols.columns] = data[numeric_cols.columns].fillna(means)
len(data),data.columns

In [ ]:
duplicates = data.duplicated(subset=list(data.columns)[1:], keep='first')
df_duplicates = data[duplicates]
df_duplicates


In [ ]:
# 去除重复数据
data.drop_duplicates(subset=list(data.columns)[1:], keep='first', inplace=True)

In [ ]:
data.head()

In [ ]:
# 提取单数行（偶数索引）和双数行（奇数索引）的pH值
even_index_pH = df_duplicates.iloc[::2]['ph']  # 偶数索引行
odd_index_pH = df_duplicates.iloc[1::2]['ph']  # 奇数索引行

# 创建折线图
plt.figure(figsize=(10, 6))
plt.plot(even_index_pH.index, even_index_pH, label='Even Index Rows')
plt.plot(odd_index_pH.index, odd_index_pH, label='Odd Index Rows')

# 添加图例
plt.legend()

# 添加标题和标签
plt.title('Comparison of pH Values in Even and Odd Rows')
plt.xlabel('Index')
plt.ylabel('pH Value')

# 显示图表
plt.show()

In [ ]:
# 改写分类字段的类型
data['DL'] = data['DL'].astype("category")
data['DZ'] = data['DZ'].astype("category")
data['SlopeClass'] = data['SlopeClass'].astype("category")

In [ ]:
data.dtypes

In [ ]:
data.columns

In [ ]:
X = data[['DEM', 'AnalyticalHillshading', 'Aspect',
       'ChannelNetworkBaseLevel', 'ChannelNetworkDistance',
       'ClosedDepressions', 'ConvergenceIndex', 'LSFactor', 'MRRTF', 'MRVBF',
       'PlanCurvature', 'ProfileCurvature', 'RelativeSlopePosition', 'Slope',
       'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth',
       'NIGHT2022', 'ETP2022_mean', 'TMP2022_mean', 'PRE2022_mean',
       'PRE2022_3', 'PRE2022_11', 'ETP2022_3', 'ETP2022_11', 'TMP2022_3',
       'TMP2022_11', 'evi', 'lswi', 'mndwi', 'ndmi', 'ndvi', 'ndwi', 'PCA_0',
       'PCA_1', 'savi', 'vari', 'DL', 'DZ', 'LON', 'LAT', 'SlopeClass']]
y = data['ph']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
# 输出训练集和测试集的形状
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
# 定义随机森林超参数的取值范围
param_dist = {
    'n_estimators': np.arange(10, 1000, 10),
    'max_features': [1.0],
    'max_depth': [None] + list(np.arange(1, 28)),
    'min_samples_split': np.arange(2, 21),
    'min_samples_leaf': np.arange(1, 21),
    'bootstrap': [True, False]
}

# 创建随机森林回归器
clf = RandomForestRegressor()

# 使用RandomizedSearchCV来寻找最佳参数
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=100, cv=5, verbose=1, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

# 打印最佳参数
print('Best Parameters: \n', random_search.best_params_)

# 使用最佳参数对测试集进行评估
best_clf = random_search.best_estimator_
score = best_clf.score(X_test, y_test)
print('Test accuracy:', score)


In [ ]:
# 使用最优参数训练RandomForestRegressor模型
rf = RandomForestRegressor(n_estimators=90,criterion='squared_error', min_samples_split=6, min_samples_leaf= 8, max_features=1.0, max_depth=21, bootstrap= True)
rf.fit(X_train,y_train)
y_test_pred = rf.predict(X_test)
y_train_pred = rf.predict(X_train)
r2_score(y_test,y_test_pred),r2_score(y_train,y_train_pred)

In [ ]:
r2 = r2_score(y_test,y_test_pred)
# 画图
plt.scatter(y_test, y_test_pred, c='b', alpha=0.5)

fit = np.polyfit(y_test, y_test_pred,deg=1)
fit_fn = np.poly1d(fit) 
plt.plot(y_test, fit_fn(y_test), c='r')

plt.xlim([min(y_test)-0.5, max(y_test)+0.5])
plt.ylim([min(y_test_pred)-0.5, max(y_test_pred)+0.5])
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('R^2: %.2f' % r2)
plt.grid()

plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(range(len(y_test[:100])),y_test[:100],c='r',label='True value')
plt.plot(range(len(y_test_pred[:100])),y_test_pred[:100],c='c',label = 'Prediction value')
plt.legend()
plt.show()

In [ ]:
mse_score = mean_squared_error(y_test, y_test_pred)
mae_score = mean_absolute_error(y_test, y_test_pred)
mape_score = mean_absolute_percentage_error(y_test, y_test_pred)
rmse_score = root_mean_squared_error(y_test, y_test_pred)
r2score = r2_score(y_test, y_test_pred)
print('Mse:', mse_score,'Mae',mae_score,'Mape',mape_score,'Rmse',rmse_score,'r2score',r2score)


In [ ]:
a = rf.feature_importances_
a

In [ ]:
# 绘制特征重要性柱状图
plt.figure(figsize=(10, 8.5))
plt.barh(X.columns, a)
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.title('Feature Importance')
plt.show()

In [ ]:
# 数据标准化后再训练
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler



# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 实例化标准化器

# scaler = StandardScaler()  # Z得分标准化（Standard Scaling）:将数据按属性（按列进行）减去其均值，并除以其标准差。结果的分布将具有均值为 0 和标准差为 1。

scaler = MinMaxScaler()  # 最小-最大标准化（Min-Max Scaling）:将所有特征缩放到 [0, 1] 范围内，或者是其他指定的范围。对异常值非常敏感。

# 加载数据
# scaler = RobustScaler()  # 稳健标准化（Robust Scaling）:使用四分位数范围来缩放数据，因此它对异常值不敏感。
for model in [StandardScaler(),MinMaxScaler(),RobustScaler()]:
    scaler = model

    # 对训练数据进行拟合和转换
    X_train_scaled = scaler.fit_transform(X_train)

    # 对测试数据进行转换
    X_test_scaled = scaler.transform(X_test)

    # 训练随机森林模型
    rf.fit(X_train_scaled, y_train)

    # 模型评估（例如，使用 R2 分数）
    r2score = rf.score(X_test_scaled, y_test)
    print("R2 Score: ", r2score)



In [ ]:
# 克里金残差训练
# 计算残差
residuals_test =y_train - y_train_pred
# 克里金残差测试
OK = OrdinaryKriging(X_train['LON'], X_train['LAT'], residuals_test, variogram_model='spherical')  #variogram_model:linear,gaussian,exponential,spherical
kriging_predictions_test, _ = OK.execute('points', X_test['LON'], X_test['LAT'])
predictions_test = y_test_pred + kriging_predictions_test
# 计算R2
r2 = r2_score(y_test, predictions_test)
r2


In [ ]:
# 计算R2
r2 = r2_score(y_test, y_test_pred)

r2

In [ ]:
# 递归特征消除 (选择最佳组合特征)
from sklearn.feature_selection import RFE,RFECV

# RFE
selector = RFECV(RandomForestRegressor(n_jobs=4),step=1,cv=5,n_jobs=4)
selector = selector.fit(X_train, y_train)

# 查看选中的特征
selected_features = selector.support_
# 计算测试集的 R2 分数
y_pred = selector.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("Selected Features: ", selected_features)
print("Number of Selected Features: ", selector.n_features_)
print("R2 Score: ", r2)

In [ ]:
selected_features,selector.n_features_

In [ ]:
# 迭代优化 (选择最佳组合特征)

best_score = 0
best_features = None

# 尝试不同数量的特征
for i in range(1, X_train.shape[1] + 1):
    # RFE 选择特征
    selector = RFE(RandomForestRegressor(n_jobs=4), n_features_to_select=i, step=1)
    selector = selector.fit(X_train, y_train)

    # 预测并计算 R2 分数
    y_pred = selector.predict(X_test)
    score = r2_score(y_test, y_pred)

    # 更新最佳分数和特征
    if score > best_score:
        best_score = score
        best_features = selector.support_

print("Best R2 Score: ", best_score)
print("Best Features: ", best_features)

In [ ]:
a = [True,  True, False, False,  True, False, False, False,  True,  True,  True,  True,
 False, False,  True, False, False, False, False, False, False, False,  True,  True,
 False, False,  True,  True, False, False, False,  True,  True,  True]
features_list = list(data.columns)
features_list.remove('pH')

In [ ]:
features = [features_list[index] for index, item in enumerate(a) if item == True]


In [ ]:
print(features),len(features)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import RFECV
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from pykrige.ok import OrdinaryKriging

def regression_prediction_cc(df, label_col, feature_cols, coord_cols, param_grid, save_model_path):
    # 分离特征和标签
    X = df[feature_cols + coord_cols]
    y = df[label_col]
    
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 初始化随机森林回归器
    rf = RandomForestRegressor(random_state=42)
    
    # 迭代特征优化
    selector = RFECV(rf, step=1, cv=5)
    selector = selector.fit(X_train[feature_cols], y_train)
    X_train_selected = selector.transform(X_train[feature_cols])
    X_test_selected = selector.transform(X_test[feature_cols])
    
    # 获取选择的特征
    selected_features = np.array(feature_cols)[selector.support_]
    print(f"Selected features: {selected_features}")
    
    # 初步超参数搜索（随机搜索）
    n_iter_search = 50
    random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=n_iter_search, cv=5, random_state=42, n_jobs=-1, verbose=2)
    random_search.fit(X_train_selected, y_train)
    
    # 基于随机搜索结果的超参数范围
    best_params = random_search.best_params_
    print(f"Random search best params: {best_params}")
    
    param_grid_fine = {
        'n_estimators': [max(10, best_params['n_estimators'] - 50), best_params['n_estimators'], min(1000, best_params['n_estimators'] + 50)],
        'max_depth': [max(1, best_params['max_depth'] - 5), best_params['max_depth'], best_params['max_depth'] + 5],
        'min_samples_split': [max(2, best_params['min_samples_split'] - 2), best_params['min_samples_split'], best_params['min_samples_split'] + 2],
        'min_samples_leaf': [max(1, best_params['min_samples_leaf'] - 1), best_params['min_samples_leaf'], best_params['min_samples_leaf'] + 1]
    }
    
    # 精细超参数搜索（网格搜索）
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid_fine, cv=5, n_jobs=-1, verbose=2)
    grid_search.fit(X_train_selected, y_train)
    
    # 最优参数
    best_params = grid_search.best_params_
    best_rf = grid_search.best_estimator_
    
    # 特征重要性
    feature_importances = best_rf.feature_importances_
    importance_df = pd.DataFrame({
        'Feature': selected_features,
        'Importance': feature_importances
    }).sort_values(by='Importance', ascending=False)
    
    # 保存特征重要性
    importance_df.to_csv(os.path.join(os.path.dirname(save_model_path), 'feature_importance.csv'), index=False)
    
    # 绘制特征重要性图
    plt.figure(figsize=(10, 6))
    plt.barh(importance_df['Feature'], importance_df['Importance'])
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.title('Feature Importances')
    plt.gca().invert_yaxis()
    plt.savefig(os.path.join(os.path.dirname(save_model_path), 'feature_importance_importance.png'))
    plt.show()
    
    # 预测与评估
    y_train_pred = best_rf.predict(X_train_selected)
    y_test_pred = best_rf.predict(X_test_selected)
    
    r2_rf = r2_score(y_test, y_test_pred)
    mae_rf = mean_absolute_error(y_test, y_test_pred)
    mse_rf = mean_squared_error(y_test, y_test_pred)
    rmse_rf = np.sqrt(mse_rf)
    
    # 输出随机森林评估分数
    print(f"Random Forest R2: {r2_rf}")
    print(f"Random Forest MAE: {mae_rf}")
    print(f"Random Forest MSE: {mse_rf}")
    print(f"Random Forest RMSE: {rmse_rf}")
    
    # 计算残差
    residuals_train = y_train - y_train_pred
    
    # 克里金残差训练
    OK = OrdinaryKriging(X_train[coord_cols[0]], X_train[coord_cols[1]], residuals_train, variogram_model='spherical')
    kriging_predictions_test, _ = OK.execute('points', X_test[coord_cols[0]], X_test[coord_cols[1]])
    
    # 最终预测
    predictions_test = y_test_pred + kriging_predictions_test
    
    # 计算克里金残差评估分数
    r2_rk = r2_score(y_test, predictions_test)
    mae_rk = mean_absolute_error(y_test, predictions_test)
    mse_rk = mean_squared_error(y_test, predictions_test)
    rmse_rk = np.sqrt(mse_rk)
    
    # 输出克里金残差评估分数
    print(f"Regression Kriging R2: {r2_rk}")
    print(f"Regression Kriging MAE: {mae_rk}")
    print(f"Regression Kriging MSE: {mse_rk}")
    print(f"Regression Kriging RMSE: {rmse_rk}")
    # 绘制随机森林评估分数图
    plt.figure(figsize=(10, 6))
    metrics = ['R2', 'MAE', 'MSE', 'RMSE']
    values_rf = [r2_rf, mae_rf, mse_rf, rmse_rf]
    values_rk = [r2_rk, mae_rk, mse_rk, rmse_rk]

    plt.subplot(2, 1, 1)
    plt.bar(metrics, values_rf)
    plt.ylabel('Scores')
    plt.title('Random Forest Evaluation Scores')

    plt.subplot(2, 1, 2)
    plt.bar(metrics, values_rk)
    plt.xlabel('Metrics')
    plt.ylabel('Scores')
    plt.title('Regression Kriging Evaluation Scores')

    plt.tight_layout()
    plt.savefig(os.path.join(os.path.dirname(save_model_path), 'evaluation_scores.png'))
    plt.show()
    # 保存模型
    with open(save_model_path, 'wb') as f:
        pickle.dump(best_rf, f)
    
    return {
        "SelectedFeatures": selected_features,
        "FeatureImportance": importance_df,
        "RandomForest": {
            "R2": r2_rf,
            "MAE": mae_rf,
            "MSE": mse_rf,
            "RMSE": rmse_rf
        },
        "RegressionKriging": {
            "R2": r2_rk,
            "MAE": mae_rk,
            "MSE": mse_rk,
            "RMSE": rmse_rk
        }
    }



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import RFECV
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from pykrige.ok import OrdinaryKriging

def regression_prediction(df, label_col, feature_cols, coord_cols, param_grid, save_model_path):
    # 分离特征和标签
    X = df[feature_cols + coord_cols]
    y = df[label_col]
    
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 初始化随机森林回归器
    rf = RandomForestRegressor(random_state=42)
    
    # 迭代特征优化
    selector = RFECV(rf, step=1, cv=5)
    selector = selector.fit(X_train[feature_cols], y_train)
    
    # 获取选择的特征索引
    selected_features =  np.array(feature_cols)[selector.support_]
    print(f"Selected features: {selected_features}")
    
    X_train_selected = selector.transform(X_train[feature_cols])
    X_test_selected = selector.transform(X_test[feature_cols])
    print(X_train_selected.shape)
    # 初步超参数搜索（随机搜索）
    random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, cv=5, n_iter=100, random_state=42, n_jobs=-1, verbose=1)
    random_search.fit(X_train_selected, y_train)
    
    # 基于随机搜索结果的超参数范围
    best_params = random_search.best_params_
    print(f"Random search best params: {best_params}")
    
    param_grid_fine = {
        'n_estimators': [max(10, best_params['n_estimators'] - 50), best_params['n_estimators'], min(1000, best_params['n_estimators'] + 50)],
        'max_depth': [max(1, best_params['max_depth'] - 5), best_params['max_depth'], best_params['max_depth'] + 5],
        'min_samples_split': [max(2, best_params['min_samples_split'] - 2), best_params['min_samples_split'], best_params['min_samples_split'] + 2],
        'min_samples_leaf': [max(1, best_params['min_samples_leaf'] - 1), best_params['min_samples_leaf'], best_params['min_samples_leaf'] + 1]
    }
    
    # 精细超参数搜索（网格搜索）
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid_fine, cv=5, n_jobs=-1, verbose=2)
    grid_search.fit(X_train_selected, y_train)
    
    # 最优参数
    best_params = grid_search.best_params_
    best_rf = grid_search.best_estimator_
    
    # 特征重要性
    feature_importances = best_rf.feature_importances_
    importance_df = pd.DataFrame({
        'Feature': selected_features,
        'Importance': feature_importances
    }).sort_values(by='Importance', ascending=False)
    
    # 保存特征重要性
    importance_df.to_csv(os.path.join(os.path.dirname(save_model_path), 'feature_importance.csv'), index=False)
    
    # 绘制特征重要性图
    plt.figure(figsize=(10, 6))
    plt.barh(importance_df['Feature'], importance_df['Importance'])
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.title('Feature Importances')
    plt.gca().invert_yaxis()
    plt.savefig(os.path.join(os.path.dirname(save_model_path), 'feature_importance_importance.png'))
    plt.show()
    
    # 预测与评估
    y_train_pred = best_rf.predict(X_train_selected)
    y_test_pred = best_rf.predict(X_test_selected)
    
    r2_rf = r2_score(y_test, y_test_pred)
    mae_rf = mean_absolute_error(y_test, y_test_pred)
    mse_rf = mean_squared_error(y_test, y_test_pred)
    rmse_rf = np.sqrt(mse_rf)
    
    # 输出随机森林评估分数
    print(f"Random Forest R2: {r2_rf}")
    print(f"Random Forest MAE: {mae_rf}")
    print(f"Random Forest MSE: {mse_rf}")
    print(f"Random Forest RMSE: {rmse_rf}")
    
    # 计算残差
    residuals_train = y_train - y_train_pred
    
    # 克里金残差训练
    OK = OrdinaryKriging(X_train[coord_cols[0]], X_train[coord_cols[1]], residuals_train, variogram_model='spherical')
    kriging_predictions_test, _ = OK.execute('points', X_test[coord_cols[0]], X_test[coord_cols[1]])
    
    # 最终预测
    predictions_test = y_test_pred + kriging_predictions_test
    
    # 计算克里金残差评估分数
    r2_rk = r2_score(y_test, predictions_test)
    mae_rk = mean_absolute_error(y_test, predictions_test)
    mse_rk = mean_squared_error(y_test, predictions_test)
    rmse_rk = np.sqrt(mse_rk)
    
    # 输出克里金残差评估分数
    print(f"Regression Kriging R2: {r2_rk}")
    print(f"Regression Kriging MAE: {mae_rk}")
    print(f"Regression Kriging MSE: {mse_rk}")
    print(f"Regression Kriging RMSE: {rmse_rk}")
    
    # 绘制随机森林评估分数图
    plt.figure(figsize=(10, 6))
    metrics = ['R2', 'MAE', 'MSE', 'RMSE']
    values_rf = [r2_rf, mae_rf, mse_rf, rmse_rf]
    values_rk = [r2_rk, mae_rk, mse_rk, rmse_rk]

    plt.subplot(2, 1, 1)
    plt.bar(metrics, values_rf)
    plt.ylabel('Scores')
    plt.title('Random Forest Evaluation Scores')

    plt.subplot(2, 1, 2)
    plt.bar(metrics, values_rk)
    plt.xlabel('Metrics')
    plt.ylabel('Scores')
    plt.title('Regression Kriging Evaluation Scores')

    plt.tight_layout()
    plt.savefig(os.path.join(os.path.dirname(save_model_path), 'evaluation_scores.png'))
    plt.show()

    # 保存模型
    with open(save_model_path, 'wb') as f:
        pickle.dump(best_rf, f)
    
    return {
        "SelectedFeatures": selected_features,
        "FeatureImportance": importance_df,
        "RandomForest": {
            "R2": r2_rf,
            "MAE": mae_rf,
            "MSE": mse_rf,
            "RMSE": rmse_rf
        },
        "RegressionKriging": {
            "R2": r2_rk,
            "MAE": mae_rk,
            "MSE": mse_rk,
            "RMSE": rmse_rk
        }
    }



In [ ]:
# 读取数据
data = pd.read_csv(r"F:\cache_data\pre_property_table\dy\feature_ph_dy.csv")
print(len(data))
# 删除有缺失值的行
# data.dropna(inplace=True)
# len(data),data.columns

# 选择数值列并计算它们的均值
numeric_cols = data.select_dtypes(include=[np.number])
means = numeric_cols.mean()
# 使用均值填充每个数值列的缺失值
data[numeric_cols.columns] = data[numeric_cols.columns].fillna(means)

In [ ]:
# 用户上传的DataFrame
df = data

coord_cols = ["LON", "LAT"]
# 用户选择的标签列和特征列
label_col = "yjz"
feature_cols = ['DEM', 'AnalyticalHillshading', 'Aspect',
       'ChannelNetworkBaseLevel', 'ChannelNetworkDistance',
       'ClosedDepressions', 'ConvergenceIndex', 'LSFactor', 'MRRTF', 'MRVBF',
       'PlanCurvature', 'ProfileCurvature', 'RelativeSlopePosition', 'Slope',
       'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth',
       'NIGHT2022', 'ETP2022_mean', 'TMP2022_mean', 'PRE2022_mean',
       'PRE2022_3', 'PRE2022_11', 'ETP2022_3', 'ETP2022_11', 'TMP2022_3',
       'TMP2022_11', 'evi', 'lswi', 'mndwi', 'ndmi', 'ndvi', 'ndwi', 'PCA_0',
       'PCA_1', 'savi', 'vari', 'DL', 'DZ','SlopeClass']

# 用户指定的超参数调优范围
param_grid = {
    'n_estimators': np.arange(10, 1000, 10),
    'max_depth': [None] + list(np.arange(1, 100)),
    'min_samples_split': np.arange(2, 21),
    'min_samples_leaf': np.arange(1, 21),
}

# 用户指定的保存模型路径
save_model_path = r"C:\Users\Runker\Desktop\testrf\best_model.pkl"

In [ ]:
scores = regression_prediction(df, label_col, feature_cols,coord_cols, param_grid, save_model_path)
print(scores)


In [ ]:
scores2 = regression_prediction_cc(df, label_col, feature_cols,coord_cols, param_grid, save_model_path)
print(scores2)
